In [1]:
import pyodbc
import pandas as pd
import math as m
import datetime

In [3]:
#connection to the base
conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=ACER7\SQLEXPRESS;'
                        'Database=master;'
                        'Trusted_Connection=yes;')

In [5]:
#creating df from the loaded data
df = pd.read_sql_query('SELECT id, x, y, z, lattitude, longitude, atm_pressure, \
                       temperture as temperature, zonal_wind, meridional_wind,\
                       extvar_3 as height, extvar_56 as CO2 FROM data_mars', conn)

In [6]:
df

,id,x,y,z,lattitude,longitude,atm_pressure,temperture,zonal_wind,meridional_wind,height,CO2
0,10,-0.850651,0.525731,0.000000,0.000000,-58.282501,537.609497,199.879898,-1.038340,0.048111,1020.968140,0.963768
1,11,0.850651,-0.525731,0.000000,0.000000,121.717003,572.585266,199.455734,-1.378017,0.146964,333.579285,0.963523
2,12,-0.850651,-0.525731,0.000000,0.000000,-121.717003,383.006866,174.186417,-7.052595,6.349962,4859.559082,0.963623
3,13,-0.510382,0.052621,0.858337,59.130299,-84.113503,775.726196,174.477737,6.694823,0.183575,-3258.103760,0.965724
4,14,-0.461270,0.053213,0.885663,62.333199,-83.419296,802.746765,168.577057,7.749964,-0.290330,-3641.076904,0.966849
...,...,...,...,...,...,...,...,...,...,...,...,...
1448719,836,0.280879,0.543996,0.790680,52.249100,27.308500,743.852417,197.880539,-2.537721,-1.061587,-3878.853027,0.967177
1448720,837,0.310296,0.583159,0.750761,48.656300,28.017200,753.293152,197.016083,-1.188300,-0.620634,-4076.799072,0.965701
1448721,838,0.337429,0.620763,0.707669,45.045601,28.527300,707.495911,197.235306,-0.480899,1.631352,-3391.293945,0.964210
1448722,839,0.362164,0.656598,0.661602,41.422100,28.880100,660.161377,196.467178,-1.494964,2.534705,-2634.977051,0.963136


In [7]:
def speed_wind(row):
    w = m.sqrt(row['zonal_wind']*row['zonal_wind']+row['meridional_wind']*row['meridional_wind'])
    return w
                   
df['speed_wind'] = df.apply(lambda row: speed_wind(row), axis = 1)

In [8]:
def direction_wind(row):
    w = m.atan2(-1*row['meridional_wind'], -1*row['zonal_wind'])
    w = m.degrees(w)
    if w < 0:
        w = w + 360
    return w

df['direction_wind'] = df.apply(lambda row: direction_wind(row), axis = 1)

In [9]:
df = df.reset_index()
df = df.rename(columns={"index": "ind"})

In [10]:
def fun_index(ind):
    i = m.floor(ind/4002)+1
    return i

df['file_number'] = df['ind'].apply(lambda x: fun_index(x))

In [12]:
def fun_data(nr):
    d = datetime.datetime.strptime('2021 '+str(nr), '%Y %j')
    data = d.strftime('%Y-%m-%d')
    return data

df['date'] = df['file_number'].apply(lambda x: fun_data(x))

In [14]:
def kel_to_cel(t):
    c = t - 273.15
    return c

df['temperture'] = df.temperture.apply(lambda x: kel_to_cel(x))

In [15]:
def co2_percent(c):
    p = round(c*100,2)
    return p

df['CO2'] = df.CO2.apply(lambda x: co2_percent(x))

In [19]:
df = df.drop(columns = 'id')
df.head()

,ind,x,y,z,lattitude,longitude,atm_pressure,temperature,zonal_wind,meridional_wind,height,CO2,speed_wind,direction_wind,file_number,date
0,0,-0.850651,0.525731,0.000000,0.000000,-58.282501,537.609497,-73.270102,-1.038340,0.048111,1020.968140,96.38,1.039454,357.347124,1,2021-01-01
1,1,0.850651,-0.525731,0.000000,0.000000,121.717003,572.585266,-73.694266,-1.378017,0.146964,333.579285,96.35,1.385832,353.912478,1,2021-01-01
2,2,-0.850651,-0.525731,0.000000,0.000000,-121.717003,383.006866,-98.963583,-7.052595,6.349962,4859.559082,96.36,9.490053,318.001001,1,2021-01-01
3,3,-0.510382,0.052621,0.858337,59.130299,-84.113503,775.726196,-98.672263,6.694823,0.183575,-3258.103760,96.57,6.697339,181.570682,1,2021-01-01
4,4,-0.461270,0.053213,0.885663,62.333199,-83.419296,802.746765,-104.572943,7.749964,-0.290330,-3641.076904,96.68,7.755400,177.854582,1,2021-01-01


In [20]:
path = 'C:\projects\Mars_weather'

In [21]:
df.to_csv(path + '\df.csv')